In [1]:
from paperlab.vae import exp
from paperlab.core import Config, ExpRunner

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# The affect of dimensionality of latent space


In [2]:
# process the experiment results
def process_exp_results(exp_results):
    train_stats, test_stats = [], []
    for result in exp_results:
        _, train_elbos, test_elbos = result

        train_stats.append(train_elbos)
        test_stats.append(test_elbos)
    
    return pd.DataFrame(train_stats).mean(), pd.DataFrame(test_stats).mean()

In [7]:
basic_params = {
    'model.dim_latent': 2,
    'model.dim_input': 28 * 28,
    'model.dim_hidden': 10,
    'learning.batch_size': 100,
    'learning.lr': 0.02,
    'learning.num_epoch': 10,
}
config = Config(**basic_params)

# change latent space from 3 to 200
stats = {}
for dim_latent in [3, 5, 10, 20]:
    config.model.dim_latent = dim_latent
    runner = ExpRunner(exp_func=exp, exp_config={'config': config}, repeat_num=5)
    runner.run()
    
    train_elbo_cruve, test_elbo_cruve = process_exp_results(runner.exp_results)
    stats[dim_latent] = train_elbo_cruve, test_elbo_cruve

experiment starts ...
config:
	 config = {'model': {'dim_latent': 3, 'dim_input': 784, 'dim_hidden': 10}, 'learning': {'batch_size': 100, 'lr': 0.02, 'num_epoch': 10}}


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(ncols=5, figsize=(30, 5), sharey=True, gridspec_kw={'wspace':0.1})

for i, dim_latent in enumerate(stats):
    train_elbo_cruve, test_elbo_cruve = stats[dim_latent]
    
    ax[i].plot(train_elbo_cruve.index, train_elbo_cruve.values, label='train')
    ax[i].plot(test_elbo_cruve.index, test_elbo_cruve.values, label='test')
    ax[i].set_ylim(-150, -100)
    ax[i].set_xscale('log')
    ax[i].set_title(f"Nz={dim_latent}")

ax[0].set_xlabel('# Training samples evaluated')
ax[0].set_ylabel('elbo')
ax[0].legend()

# Learned MNIST Manifold
when we set the size of latent space as 2, the corresponding image generated by the given is shown below.

In [ ]:
basic_params = {
    'model.dim_latent': 2,
    'model.dim_input': 28 * 28,
    'model.dim_hidden': 10,
    'learning.batch_size': 100,
    'learning.lr': 0.02,
    'learning.num_epoch': 10,
}
config = Config(**basic_params)
model, *_ = exp(config)

In [ ]:
num_row, num_col = 20, 20

# draw samples from encoder 
xs, ys = np.meshgrid(np.linspace(-5, 5, num_row), np.linspace(-5, 5, num_col))
z = np.stack([xs.flatten(), ys.flatten()], axis=1)
draw_samples = model.decoder.prob(torch.tensor(z, dtype=torch.float))
imgs = draw_samples.data.numpy().reshape(-1, 28, 28)


# show the samples
f, ax = plt.subplots(num_row, num_col, 
                     figsize=(10, 10),
                     gridspec_kw={'wspace':0, 'hspace':0}, 
                     squeeze=True)

for i in range(num_row):
    for j in range(num_col):
        im = imgs[num_row * i + j]
        ax[i, j].imshow(im, cmap='Greys', aspect='auto')
        ax[i, j].axis('off')